---

## Obsah
1. [ER Diagram](#er-diagram)  
2. [Normální formy (1NF, 2NF, 3NF)](#normalni-formy)  
3. [Rozklad N–M: spojovací tabulky](#rozklad-n-m)  
4. [Domény (datové typy)](#domeny)  
5. [CREATE TABLE: constraints a domény](#create-table)  
6. [SELECT](#select)   
   1. [FROM](#from)  
   2. [JOIN (INNER, OUTER)](#join)  
   3. [WHERE (včetně regulárních výrazů)](#where)  
   4. [ORDER BY (sekundární řazení)](#order-by)  
   5. [SELECT (projekce)](#projekce)  
7. [GROUP BY](#group-by)  
   - [Typy sloupců: klíče, hodnoty, kategorie](#typy-sloupcu)  
8. [Transakce (ACID)](#transakce)  
   1. [Atomicita](#atomicita)  
   2. [Konzistence](#konzistence)  
   3. [Izolovanost (zámky, multivariantní kontrola)](#izolovanost)  
   4. [Trvalost](#trvalost)  
9. [Efektivita a indexy](#efektivita)  
   1. [Typy indexových struktur](#indexove-struktury)  
   2. [Join algoritmy](#join-algoritmy)  
   3. [Jak urychlit SELECT (JOIN, selection, ORDER BY)](#urychleni)  
   4. [Co zpomaluje INSERT](#zpomalení)  
10. [VIEW](#view)  
    1. [Předpřivaný dotaz](#predpripraveny-dotaz)  
    2. [Ochrana dat](#ochrana-dat)  
    3. [Modifikovatelné pohledy](#modifikovatelne-view)  
    4. [Materializované pohledy](#materializovane-view)  


---

<a name="er-diagram"></a>
## 1. ER Diagram
- **Definice**: Entity-Relationship Diagram (ERD) je **grafické znázornění** struktury databáze: entit (tabulek), jejich atributů (sloupců) a vztahů mezi entitami.
- **Účel**:
  - Vizualizovat návrh databáze před implementací.
  - Ověřit správnost relací (1:1, 1:N, M:N) a kardinality (např. `zero n`, `one only`).
  - Slouží jako dokumentace pro vývojáře i analytiiky.
- **Příklad (filmová databáze)**:
  - Entita `Films` má atributy `Film_ID`, `Title`, `Release_Year`, `Genre_ID`, `Director_ID`, `Duration`, `Rating`.
  - Entita `Genres` má `Genre_ID`, `Genre_Name`.
  - Mezi `Films` a `Genres` je vztah **N:1** (jeden žánr může mít víc filmů, film patří právě jednomu žánru).
  - Entita `Directors` má `Director_ID`, `First_Name`, `Last_Name`, `Birthdate`.
  - Mezi `Films` a `Directors` je vztah **N:1** (jeden režisér má víc filmů, film má právě jednoho režiséra).
  - Entita `Actors` a spojovací entita `Film_Actors` (pro vztah M:N).
- **Typy vztahů**:
  1. **1:1 (one-to-one)** – vzácné, např. každý uživatel má právě jednu profilovou stránku.
  2. **1:N (one-to-many)** – např. každý režisér (`Directors`) může natočit více filmů (`Films`).
  3. **N:1 (many-to-one)** – stejně jako 1:N, jen z opačného pohledu: více filmů patří jednomu režisérovi.
  4. **M:N (many-to-many)** – např. herci (`Actors`) hrají ve více filmech (`Films`) a naopak; řeší se pomocí spojovací tabulky `Film_Actors`.


<a name="normalni-formy"></a>
## 2. Normální formy (1NF, 2NF, 3NF)
- **Účel normalizace**: Odstranit redundanci, zajistit integritu dat, minimalizovat anomálie při vkládání/mazání.
- **První normální forma (1NF)**:
  - **Pravidlo**: Každý atribut musí obsahovat **atomické hodnoty** (nelze mít seznam/hromadná data v jednom sloupci).
  - **Příklad**:  
    - Ne: `Actors.Name = 'Tom Hanks, Denzel Washington'` (dva herci v jednom sloupci).  
    - Ano: dva řádky (jeden s `Tom Hanks`, druhý s `Denzel Washington`).
- **Druhá normální forma (2NF)**:
  - **Pravidlo**:  
    1. Splňovat 1NF.  
    2. Každý neklíčový atribut závisí na *celém* primárním klíči, ne jen na části (aplikuje se na tabulky s kombinovaným PK).
  - **Příklad**:  
    - Mějme spojovací tabulku `Film_Actors(Film_ID, Actor_ID, Role)`.  
      - Primární klíč = `(Film_ID, Actor_ID)`.  
      - Atribut `Role` závisí na obou (na spojené dvojici film–herec), tudíž 2NF je splněna.
- **Třetí normální forma (3NF)**:
  - **Pravidlo**:  
    1. Splňovat 2NF.  
    2. Žádný neklíčový atribut nesmí záviset transitivně na primárním klíči (tzn. neexistuje závislost „A → B → C“ pro neklíčové atributy B a C, kde B i C nejsou klíč).
  - **Příklad**:  
    - Tabulka `Films(Film_ID, Title, Director_ID, Director_Name)`.  
      - Pokud uložíme i `Director_Name`, závisí na `Director_ID`, ale `Director_ID` závisí na `Film_ID`: transitive dependency → porušení 3NF.  
    - Správně je jen `Director_ID` (FK). `Director_Name` bude v tabulce `Directors`.
- **Shrnutí**:  
  - **1NF**: rozklad vícenásobných hodnot na atomy.  
  - **2NF**: odstranění částečných závislostí.  
  - **3NF**: odstranění tranzitivních závislostí.


<a name="rozklad-n-m"></a>
## 3. Rozklad N–M: spojovací tabulky
1. **Definice**:  
   - **M:N vztah** znamená, že každý záznam v tabulce A může být spojen s mnoha záznamy v tabulce B a naopak.  
   - Abychom to udrželi v relačním modelu, vytvoří se třetí tabulka (nazvěme ji `A_B`), která obsahuje odkazy na primární klíče obou tabulek.
2. **Příklad (filmová databáze)**:  
   - Tabulky: `Films` a `Actors`.  
   - Film může mít více herců, herec může hrát ve více filmech.  
   - Spojovací tabulka:  
     ```sql
     CREATE TABLE Film_Actors (
       Film_ID INT REFERENCES Films(Film_ID) ON DELETE CASCADE,
       Actor_ID INT REFERENCES Actors(Actor_ID) ON DELETE CASCADE,
       Role VARCHAR(100),
       PRIMARY KEY (Film_ID, Actor_ID)
     );
     ```
3. **Příklad z jiného oboru**:  
   - **Student–Course**: `Students` a `Courses` mají M:N → `Enrollments(Student_ID, Course_ID, Enrollment_Date)`.  
   - Každý student může být zapsaný v několika kurzech, kurz má mnoho studentů.
4. **Důvody pro spojovací tabulku**:  
   - Zachování **referenční integrity** (FK záznamy odkazují na existující záznamy v rodičovských tabulkách).  
   - Možnost **uchovávat atributy vztahu** (např. datum zápisu, role v projektu, hodnocení, …).  
   - Normalizace (odstranění redundance, 3NF).


<a name="domeny"></a>
## 4. Domény (datové typy)
- **Definice**:  
  Doména je **množina povolených hodnot** pro sloupec v tabulce. V SQL se doména určuje datovým typem (`INT`, `VARCHAR`, `DATE`, `DECIMAL`, …) a případně jeho omezeními (`CHECK`, rozsah, délka).
- **Příklad rozdílných datových typů**:  
  1. **Real (double precision)** vs. **numeric (decimal)**  
     - `REAL` / `DOUBLE PRECISION`:  
       - 32bitové / 64bitové plovoucí desetinné číslo.  
       - **Rychlé** pro matematické operace, ale **ne přesné** (zaokrouhlovací chyby).  
       - Vhodné pro vědecké výpočty (měření, teploty, fyzikální experimenty).  
     - `NUMERIC(p, s)` / `DECIMAL(p, s)`:  
       - Fixní přesnost, kde `p` je počet číslic celkem, `s` je počet desetinných míst.  
       - **Přesné** – žádné zaokrouhlovací chyby.  
       - Vhodné pro peníze, finanční výpočty, hodnocení (např. `Rating DECIMAL(3,1)` = mezi `0.0` a `10.0`).
  2. **VARCHAR(n)** vs. **TEXT**  
     - `VARCHAR(n)`: řetězec délky max. `n`.  
     - `TEXT`: libovolná délka, žádné pevné omezení (v PostgreSQL pod kapotou „neomezené“).  
     - Pokud očekáváš omezeně dlouhé hodnoty (např. `Username VARCHAR(50)`), použij `VARCHAR(50)`.
- **Další běžné typy**:  
  - `INT` nebo `BIGINT` (celá čísla).  
  - `DATE`, `TIMESTAMP` (datum, datum + čas).  
  - `BOOLEAN` (pravda/nepravda).  
  - `BYTEA` (binární data, např. obrázky).  
- **Příklad z jiné domény**:  
  - **Bankovní účet**: sloupec `Balance NUMERIC(12,2)` = přesné peněžní částky až do bilionů s dvěma desetinnými místy.
- **Důležitost domén**:  
  - **Omezení rozsahu** → prevence chyb (nelze do sloupce s penězi vložit text).  
  - **Úspora místa** (např. `SMALLINT` místo `INT`, pokud hodně malých celých čísel).  
  - **Výkon** – správné typy dovolují optimalizaci indexů a operací.


<a name="create-table"></a>
## 5. CREATE TABLE: constraints a domény
1. **Syntaxe**:  
   ```sql
   CREATE TABLE TableName (
     Column1 DataType [CONSTRAINTS],
     Column2 DataType [CONSTRAINTS],
     ...,
     [PRIMARY KEY (ColumnX)],
     [FOREIGN KEY (ColumnY) REFERENCES OtherTable(OtherColumn) ON DELETE CASCADE],
     [UNIQUE (ColumnZ)],
     [CHECK (condition)]
   );
   ```
2. **Domény**:  
   - Zvolíme datový typ dle povahy atributu.  
   - Např.  
     ```sql
     Film_ID      SERIAL,  
     Title        VARCHAR(200) NOT NULL,  
     Release_Year INT CHECK (Release_Year >= 1888),  -- první film roku 1888
     Rating       DECIMAL(3,1) CHECK (Rating BETWEEN 0.0 AND 10.0),
     Duration     INT        CHECK (Duration > 0)
     ```
3. **Omezení (constraints)**:  
   - **PRIMARY KEY**: jednoznačně identifikuje každý řádek.  
   - **FOREIGN KEY**: referenční integrita → hodnota sloupce musí existovat v referencované tabulce.  
   - **UNIQUE**: zajišťuje jedinečnost hodnot ve sloupci/sloupcích.  
   - **NOT NULL**: sloupec musí mít hodnotu, nelze vložit `NULL`.  
   - **CHECK**: ručně definované omezení (např. rozsah, formát).
4. **Příklad tabulky `Films`**:
   ```sql
   CREATE TABLE Films (
     Film_ID      SERIAL PRIMARY KEY,
     Title        VARCHAR(200) NOT NULL,
     Release_Year INT NOT NULL CHECK (Release_Year >= 1888),
     Genre_ID     INT REFERENCES Genres(Genre_ID) ON DELETE SET NULL,
     Director_ID  INT REFERENCES Directors(Director_ID) ON DELETE SET NULL,
     Duration     INT NOT NULL CHECK (Duration > 0),
     Rating       DECIMAL(3,1) CHECK (Rating BETWEEN 0.0 AND 10.0)
   );
   ```
5. **Význam omezení**:  
   - Zajišťují **kvalitu dat** (validita, konzistence).  
   - Pomáhají odhalit chyby už při vložení (např. `Rating = 15.0` nelze).  
   - ON DELETE CASCADE / SET NULL → chování při mazání rodiče, zamezuje „visícím“ odkazům.


<a name="select"></a>
## 6. SELECT


### 6.1 <a name="from"></a> FROM
- **Definice**: Základní klauzule, určuje **hlavní tabulku** nebo **výchozí zdroj** dat.  
- **Syntaxe**:
  ```sql
  SELECT columns
  FROM TableName
  ```
- **Příklad**:
  ```sql
  SELECT Title, Release_Year
  FROM Films;
  ```
- **Více tabulek**:  
  - Můžeme za `FROM` uvést více tabulek rozdělených čárkami, ale pak se implicitně použije `CROSS JOIN`. Doporučuje se explicitní `JOIN`.


### 6.2 <a name="join"></a> JOIN (INNER, OUTER)
- **Účel**: Spojit řádky z více tabulek na základě vzájemné podmínky (klíčů).  
- **Typy**:
  1. **INNER JOIN**  
     - Vrátí **pouze záznamy**, které mají odpovídající shodu ve všech spojených tabulkách.  
     - **Příklad** (filmová DB):
       ```sql
       SELECT Films.Title, Directors.First_Name || ' ' || Directors.Last_Name AS Director
       FROM Films
       INNER JOIN Directors ON Films.Director_ID = Directors.Director_ID;
       ```
     - V jiném oboru:  
       - Tabulky `Employees` a `Departments`.  
       - `INNER JOIN` vrátí jen zaměstnance, kteří jsou přiřazeni k existujícímu oddělení.  
       ```sql
       SELECT E.Name, D.DepartmentName
       FROM Employees E
       INNER JOIN Departments D ON E.DepartmentID = D.DeptID;
       ```
  2. **LEFT OUTER JOIN (nebo jen LEFT JOIN)**  
     - Vrátí **všechny řádky z levé tabulky** a **připojí odpovídající** z pravé, pokud existují.  
     - Pokud **žádná shoda neexistuje**, pravé sloupce budou `NULL`.  
     - **Příklad (filmová DB)**:
       ```sql
       SELECT Films.Title, Reviews.Rating
       FROM Films
       LEFT JOIN Reviews ON Films.Film_ID = Reviews.Film_ID;
       ```
       - Zobrazí všechny filmy, a pokud mají recenzi, připojí `Rating`. Jinak bude `Rating = NULL`.
     - V jiném oboru:  
       - `Customers` a `Orders`.  
       - Chceme všechny zákazníky, i když nikdy nic neobjednali:
       ```sql
       SELECT C.Name, O.OrderID
       FROM Customers C
       LEFT JOIN Orders O ON C.CustomerID = O.CustomerID;
       ```
  3. **RIGHT OUTER JOIN (nebo RIGHT JOIN)**  
     - Opačně: vrátí **všechny řádky z pravé tabulky** a přidá odpovídající z levé, pokud existují, jinak `NULL`.  
     - Méně často používané (lze nahradit `LEFT JOIN` prohozením tabulek).  
     - **Příklad** (obchodní DB):
       ```sql
       SELECT C.Name, O.OrderID
       FROM Customers C
       RIGHT JOIN Orders O ON C.CustomerID = O.CustomerID;
       ```
       - Zobrazí všechny `Orders`, i když některý `CustomerID` v `Customers` neexistuje.
  4. **FULL OUTER JOIN**  
     - Vrátí **všechny řádky z obou tabulek**; pokud neexistuje shoda, příslušný sloupec je `NULL`.  
     - **Příklad** (doklady vs. objednávky):
       ```sql
       SELECT A.DocID, B.OrderID
       FROM Invoices A
       FULL OUTER JOIN Orders B ON A.OrderID = B.OrderID;
       ```
       - Zobrazí všechny faktury i objednávky, ať už se k sobě vztahují, nebo ne.
- **Klíčové body**:
  - `INNER JOIN` = **průnik** (intersection).  
  - `LEFT JOIN` = **levý spoj** (alespoň levá strana).  
  - `RIGHT JOIN` = **pravý spoj** (alespoň pravá strana).  
  - `FULL JOIN` = **sjednocení** s `NULL` v chybějících polích.


### 6.3 <a name="where"></a> WHERE (včetně regulárních výrazů)
- **Účel**: Filtrovat řádky na základě podmínky.  
- **Syntaxe**:
  ```sql
  SELECT columns
  FROM TableName
  WHERE condition;
  ```
- **Operátory**:
  - Porovnávací: `=`, `<>`, `<`, `>`, `<=`, `>=`  
  - Logické: `AND`, `OR`, `NOT`  
  - Pattern matching: `LIKE`, `ILIKE` (PostgreSQL – ignoruje velikost písmen), `SIMILAR TO`, `~` (regex match), `~*` (case-insensitive regex)  
- **Příklad (filmová DB)**:
  ```sql
  SELECT Title 
  FROM Films 
  WHERE Title ILIKE '%dark%' AND Release_Year >= 2000;
  ```
  - Vrátí všechny filmy, jejichž název obsahuje „dark“ bez ohledu na velikost písmen a jsou vydány po roce 2000.
- **Regex (PostgreSQL)**:
  - `~`: full regex match, case-sensitive  
  - `~*`: full regex match, case-insensitive  
- **Příklad z jiného oboru**:
  - Tabulka `Emails(EmailID, EmailAddress)`.  
  - Chceme vyfiltrovat e-maily ze seznamu Gmail:
  ```sql
  SELECT EmailAddress
  FROM Emails
  WHERE EmailAddress ~* '^[A-Za-z0-9._%+-]+@gmail\.com$';
  ```
  - Regulární výraz `^[A-Za-z0-9._%+-]+@gmail\.com$` zajišťuje, že přesně hledáme e-maily končící `@gmail.com`.


### 6.4 <a name="order-by"></a> ORDER BY (sekundární řazení)
- **Účel**: Řadit výstup výsledku dotazu podle jednoho nebo více sloupců.  
- **Syntaxe**:
  ```sql
  SELECT columns
  FROM TableName
  ORDER BY column1 [ASC|DESC], column2 [ASC|DESC], ...;
  ```
- **Primární vs. sekundární řazení**:
  1. **Primární řazení** – určuje první sloupec, podle kterého se řadí (např. `ORDER BY Rating DESC` → nejvyšší `Rating` nejdřív).  
  2. **Sekundární řazení** – pokud mají dva záznamy stejnou primární hodnotu, použije se druhý sloupec (např. `ORDER BY Rating DESC, Title ASC` → když dva filmy mají stejný `Rating`, seřadí se je podle `Title` vzestupně).
- **Příklad (filmová DB)**:
  ```sql
  SELECT Title, Release_Year, Rating
  FROM Films
  ORDER BY Rating DESC, Release_Year ASC;
  ```
- **Příklad z jiného oboru**:
  - Tabulka `Products(ProductID, Category, Price)`.  
  - Chceme seřadit nejprve podle `Category` vzestupně, a uvnitř každé kategorie podle `Price` sestupně:
  ```sql
  SELECT Category, ProductID, Price
  FROM Products
  ORDER BY Category ASC, Price DESC;
  ```


### 6.5 <a name="projekce"></a> SELECT (projekce)
- **Definice**: **Projekce** znamená vybrat konkrétní sloupce (atributy), které chceme ve výsledku zobrazit.  
- **Syntaxe**:
  ```sql
  SELECT column1, column2, ... 
  FROM TableName;
  ```
- **Všechny sloupce**:
  - `SELECT * FROM TableName;`  
  - Nevhodné v produkčním prostředí, pokud nepotřebujeme všechny sloupce, protože se zbytečně přenáší data.
- **Výběr konkrétních sloupců**:
  - Umožňuje zrychlit dotaz i zlepšit čitelnost:  
    ```sql
    SELECT Title, Duration 
    FROM Films;
    ```
- **Alias pro sloupce**:
  - `SELECT Title AS FilmName, Rating AS Score FROM Films;`
- **Výhody projekce**:
  - Omezí množství přenesených dat.
  - Zpřehlední výstup (není třeba všechny sloupce).
  - Umožňuje použít výpočty:  
    ```sql
    SELECT Title, Duration / 60.0 AS Duration_Hours 
    FROM Films;
    ```


<a name="group-by"></a>
## 7. GROUP BY
- **Účel**: Slouží ke **skupinování řádků** na základě hodnoty jednoho nebo více sloupců a **aplikaci agregačních funkcí** na každou skupinu.  
- **Syntaxe**:
  ```sql
  SELECT columns, aggregate_function(column)
  FROM TableName
  GROUP BY columns;
  ```
- **Agregační funkce**:  
  - `COUNT()`, `SUM()`, `AVG()`, `MIN()`, `MAX()`.
- **HAVING**:  
  - Filtr pro agregované výsledky (aplikuje se až po `GROUP BY`).  
  - Např. `HAVING COUNT(*) > 5`.
- **Příklad (filmová DB)**:
  ```sql
  SELECT Genres.Genre_Name, COUNT(Films.Film_ID) AS Film_Count
  FROM Films
  INNER JOIN Genres ON Films.Genre_ID = Genres.Genre_ID
  GROUP BY Genres.Genre_Name
  HAVING COUNT(Films.Film_ID) >= 5
  ORDER BY Film_Count DESC;
  ```
- **Příklad z jiného oboru**:
  - Tabulka `Sales(OrderID, CustomerID, Amount)`.  
  - Chceme zjistit, kolik objednávek a jaký celkový obrat má každý zákazník:
  ```sql
  SELECT CustomerID, COUNT(OrderID) AS OrderCount, SUM(Amount) AS TotalSpent
  FROM Sales
  GROUP BY CustomerID
  HAVING SUM(Amount) > 1000
  ORDER BY TotalSpent DESC;
  ```


<a name="typy-sloupcu"></a>
### 7.1 Typy sloupců (klíče, hodnoty, kategorie)
- **Primární klíč (PK)**:
  - Unikátně identifikuje každý řádek (např. `Film_ID`, `User_ID`).
- **Cizí klíč (FK)**:
  - Odkazuje na primární klíč jiné tabulky (např. `Films.Genre_ID` → `Genres.Genre_ID`).
- **Atributy (hodnoty)**:
  - Sloupce, které obsahují data o entitě (např. `Title`, `Release_Year`).
- **Kategorie (klasifikační sloupce)**:
  - Sloupce, podle kterých se často agreguje nebo třídí (např. `Genre_Name` při agregaci počtu filmů na žánr).


<a name="transakce"></a>
## 8. Transakce (ACID)
- **Transakce**: Jednotka práce, která může obsahovat jednu nebo více databázových operací (INSERT, UPDATE, DELETE, SELECT).  
- **ACID** je zkratka pro vlastnosti, které transakce musí splňovat:
  1. <a name="atomicita"></a> **Atomicita (Atomicity)**  
     - „Vše nebo nic.“  
     - Buď se **všechny operace** v transakci provedou, nebo se **žádná neprovede** (v případě chyby se provede rollback).  
     - **Příklad**:  
       - V bankovním systému převádíš peníze:  
         1) Snížit zůstatek z účtu A.  
         2) Zvětšit zůstatek na účtu B.  
       - Pokud druhá operace selže, musí se vrátit i první, aby nebylo možné peníze „vypustit“.
  2. <a name="konzistence"></a> **Konzistence (Consistency)**  
     - Po dokončení transakce musí databáze zůstat v platném stavu podle definovaných pravidel a omezení.  
     - Omezení (constraints) – PK, FK, unikátnost, CHECK.  
     - **Příklad**:  
       - Nemůžeš vložit recenzi na neexistující `Film_ID`, protože cizí klíč to neumožní – konzistence zůstává zachována.
  3. <a name="izolovanost"></a> **Izolovanost (Isolation)**  
     - Transakce by měly probíhat, jako by byly izolované – záznamy se navzájem neovlivní, dokud se transakce nedokončí.  
     - **Úrovně izolace**:  
       - **Read Uncommitted** – „špinavé čtení“, lze číst data z nedokončených transakcí.  
       - **Read Committed** – výchozí úroveň v PostgreSQL; nelze číst necommitované změny.  
       - **Repeatable Read** – při opakovaném čtení v jedné transakci vidíš stejné řádky, i když mezitím jiné transakce změnily data.  
       - **Serializable** – nejpřísnější, chová se, jako by transakce běžely sekvenčně.  
     - **Multivariační kontrola (MVCC)**:  
       - PostgreSQL používá MVCC, aby poskytl izolaci bez uzamykání celé tabulky.  
       - Každá transakce vidí „verzi“ dat, která byla platná na začátku transakce, nebo vlastní změny.
  4. <a name="trvalost"></a> **Trvalost (Durability)**  
     - Jakmile je transakce potvrzena (commit), její změny jsou **trvale** uloženy na disku – i při pádu systému se obnoví.  
     - V PostgreSQL se používá WAL (Write-Ahead Logging) pro zajištění trvalosti.


<a name="efektivita"></a>
## 9. Efektivita a indexy


<a name="indexove-struktury"></a>
### 9.1 Typy indexových struktur
- **Účel indexů**:  
  - Zrychlit vyhledávání (`SELECT`), řazení (`ORDER BY`) a spojování (`JOIN`).  
  - Na úkor pomalejšího vkládání (`INSERT`), aktualizací (`UPDATE`), protože indexy se musí udržovat.
- **B-tree index (výchozí)**:  
  - Vyvážený strom, vhodný pro `=`, `>`, `<`, `BETWEEN`, `ORDER BY`.  
  - V PostgreSQL jsou B-tree indexy standardem.  
  - **Příklad**:  
    ```sql
    CREATE INDEX idx_films_title ON Films(Title);
    ```
    - Díky tomu se hledání podle `Title` (např. `WHERE Title = 'Interstellar'`) zrychlí.
- **Hash index**:  
  - Efektivní pro `=` (rovnost), ale NE pro rozsahy (`<`, `>`, `BETWEEN`).  
  - V PostgreSQL se méně často používá (méně univerzální než B-tree).
- **GIN / GiST / SP-GiST / BRIN**:  
  - Speciální indexy pro plnotextové vyhledávání, geometrické datové typy, velmi rozsáhlé tabulky (např. geografická data).  
  - **Příklad plnotextový**:  
    ```sql
    CREATE INDEX idx_actors_name ON Actors USING GIN(to_tsvector('english', First_Name || ' ' || Last_Name));
    ```


<a name="join-algoritmy"></a>
### 9.2 Join algoritmy
1. **Nested Loop Join** (vnořené cykly)  
   - Pro každý řádek z první tabulky prochází druhou tabulku a hledá shodu.  
   - Jednoduché, ale **nevhodné pro velké tabulky**, protože má složitost O(n × m).  
   - PostgreSQL automaticky volí Nested Loop, když se očekává malý počet řádků z jedné strany (např. tabulka s malým množstvím dat).
2. **Merge Join**  
   - Obě tabulky se **nejprve seřadí podle sloupce, podle kterého se spojí**, pak se paralelně procházejí.  
   - Rychlé, když jsou oba vstupy již seřazené (nebo má index).  
   - Pořadí: O(n log n + m log m) pro seřazení + O(n + m) pro samotné spojení.  
   - Výhoda: funguje dobře pro větší množství dat, když jsou oba vstupy seřazené.
3. **Hash Join**  
   - Vytvoří se **hash tabulka** pro menší tabulku (na základě klíče), poté se prochází větší tabulka a provádí se hash lookup.  
   - Výkon: O(n + m), ale potřebuje dodatečnou paměť pro hash tabulku.  
   - Vhodné, když tabulka nemá index a nejsou seřazené.
4. **Loop Indexed Join**  
   - Varianta Nested Loop, kde se druhá tabulka neprochází lineárně, ale pomocí indexu (např. `B-tree`), což snižuje složitost.  
- **Jak PostgreSQL vybírá algoritmus**:  
  - Odhaduje počet řádků (statistics) a vybere nejefektivnější způsob podle plánovače (planner).


<a name="urychleni"></a>
### 9.3 Jak urychlit SELECT (JOIN, selection, ORDER BY)
- **Indexy**:  
  - Zajistit, aby sloupce ve `WHERE`, `JOIN ON`, `ORDER BY` měly index.  
  - Příklad:  
    ```sql
    CREATE INDEX idx_films_genre ON Films(Genre_ID);
    CREATE INDEX idx_films_director ON Films(Director_ID);
    ```
- **EXPLAIN / EXPLAIN ANALYZE**:  
  - Použij `EXPLAIN` před dotazem, abys viděl, jakým způsobem PostgreSQL naplánuje dotaz.  
  - `EXPLAIN ANALYZE` navíc skutečně provede dotaz a vrátí reálné časy a počty řádků.
- **Výběr vhodného JOINu**:  
  - Pokud je jedna tabulka velmi malá, Nested Loop je v pořádku.  
  - Pokud jsou obě tabulky velké a neseřazené, Hash Join často lepší.  
  - Pokud jsou obě seřazené, Merge Join.
- **Výběr menší množiny před JOIN (predicate pushdown)**:  
  - Filtruj vnější tabulku co nejdřív, aby se snížila velikost držené datové množiny:  
    ```sql
    SELECT ...
    FROM (
      SELECT * FROM Films WHERE Release_Year >= 2010  -- úzký výběr nejprve
    ) AS recent_films
    JOIN Reviews ON recent_films.Film_ID = Reviews.Film_ID;
    ```
- **Materiálizované pohledy** (materialized views):  
  - Pokud je dotaz náročný a nemění se často, lze výsledky uložit a přiřadit index.  
  - Např. agregované statistiky, které se obvykle aktualizují jednou denně.
- **Partitioning (dílčování tabulek)**:  
  - Pro velmi velké tabulky lze použít rozdělení (partition) podle určitého sloupce (rok, oblast).  
  - PostgreSQL podporuje dělení (range partitioning, list partitioning).
- **Vyhnout se SELECT ***:  
  - Vyber pouze potřebné sloupce, aby se zamezilo přenosu nepotřebných dat.


<a name="zpomalení"></a>
### 9.4 Co zpomaluje INSERT
- **Velké objemy indexů**:  
  - Každý index se musí aktualizovat při vložení nového řádku → zpomalení.  
  - Pokud vložíš tisíce řádků, je lepší:  
    1. Dočasně **zrušit index**.  
    2. Vložit data.  
    3. **Opětovně vytvořit index** (jednou seřadí celou tabulku).
- **Spouštění triggerů**:  
  - Pokud máš před/v-after `INSERT` triggery, spouštění kódu v každé řádce může výrazně zpomalit.
- **Kontrolní omezení (CHECK, FK)**:  
  - Při každém vložení se ověřuje cizí klíč → další dotaz na referenční tabulku.  
  - Pokud se vkládá se špatně navrženými FK, může to znamenat více kontrol.
- **Velké transakce bez COMMIT**:  
  - Příliš dlouhé transakce se hůře spravují a mohou způsobit vyčerpání paměti (MVCC udržuje staré verze záznamů).
- **Diskové I/O**:  
  - Pokud se hodně zapisuje a databáze není optimalizovaná (např. WAL na SSD), může dojít k výkonovému úzkému hrdlu.


<a name="view"></a>
## 10. VIEW


### 10.1 <a name="predpripraveny-dotaz"></a> Předpřipravený dotaz (Stored Query)
- **Definice**: Pohled (view) je **pojmenovaný SELECT**, který chová jako virtuální tabulka.  
- **Výhody**:
  - Opakované použití složitých dotazů bez nutnosti znovu psát JOINy a další logiku.
  - Umožňuje snadnou čitelnost a údržbu kódu.
- **Příklad (filmová DB)**:
  ```sql
  CREATE VIEW film_director_summary AS
  SELECT 
    Films.Title,
    Directors.First_Name || ' ' || Directors.Last_Name AS Director
  FROM Films
  INNER JOIN Directors ON Films.Director_ID = Directors.Director_ID;
  ```
  - Pak stačí:  
    ```sql
    SELECT * FROM film_director_summary;
    ```


### 10.2 <a name="ochrana-dat"></a> Ochrana dat
- **Účel**:  
  - Omezit přístup k citlivým sloupcům.  
  - Například nechceme, aby externí uživatelé viděli kolik přesně film vydělal (sloupec `Budget`), ale pouze počet recenzí a průměrné hodnocení.
- **Příklad**:
  ```sql
  CREATE VIEW public_view AS
  SELECT Title, Release_Year, Rating
  FROM Films;
  ```
  - Uživatelé mohou mít právo `SELECT` na `public_view`, ale ne na celou tabulku `Films`.


### 10.3 <a name="modifikovatelne-view"></a> Modifikovatelné pohledy
- **Podmínka**:  
  - Pohled je modifikovatelný, pokud splňuje určitá kritéria (v PostgreSQL: pouze jednoduchý `SELECT` z jedné tabulky, bez agregací, bez `GROUP BY`, bez `DISTINCT`, bez spojování více tabulek).
- **Příklad modifikovatelného pohledu**:
  ```sql
  CREATE VIEW simple_film_titles AS
  SELECT Film_ID, Title
  FROM Films;
  ```
  - Na tento pohled lze provádět `INSERT`, `UPDATE`, `DELETE` (úpravy jsou přeposílány do základní tabulky `Films`).
- **Příklad nemodifikovatelného pohledu**:
  ```sql
  CREATE VIEW non_updatable AS
  SELECT Title, COUNT(*) AS Count
  FROM Films
  GROUP BY Title;
  ```
  - Nelze měnit data skrze tento pohled (agragace).


### 10.4 <a name="materializovane-view"></a> Materializované pohledy
- **Definice**:  
  - Materializovaný pohled (materialized view) je „snapshot“ výsledku SELECT v daném čase. Data jsou **fyzicky uložena** jako tabulka.
- **Výhody**:  
  - Rychlé čtení (data jsou již vypočítaná), vhodné pro náročné agregace.  
  - Lze indexovat materializovaný pohled.
- **Nevýhody**:  
  - **Nejsou automaticky aktualizovány** při změně základních tabulek.  
  - Je potřeba je občas **refreshovat**:
    ```sql
    CREATE MATERIALIZED VIEW film_rating_summary AS
    SELECT 
      Films.Title, 
      AVG(Reviews.Rating) AS Avg_Rating
    FROM Films
    INNER JOIN Reviews ON Films.Film_ID = Reviews.Film_ID
    GROUP BY Films.Title;

    -- Refresh, aby se znovu přepočítalo:
    REFRESH MATERIALIZED VIEW film_rating_summary;
    ```
- **Příklad z jiného oboru**:  
  - Systém pro sledování počasí, kde se denně agregují průměrné teploty měst.  
    - Základní tabulka: `WeatherReadings(CityID, Temperature, Timestamp)`.  
    - Materializovaný pohled:  
      ```sql
      CREATE MATERIALIZED VIEW daily_avg_temps AS
      SELECT CityID, DATE(Timestamp) AS Day, AVG(Temperature) AS AvgTemp
      FROM WeatherReadings
      GROUP BY CityID, DATE(Timestamp);
      ```
    - Data se obnovují jednou denně, aby se nemusely pokaždé měřit tisíce záznamů.
